In [1]:
# Import majot libraries for project
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import variables as var


In [2]:
def scrape_event_type(driver, url):
    driver.get(url)
    category = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, var.xpaths['yetu_cat'])))
    category.click()
    event_types = []
    for x in driver.find_elements_by_class_name("BLM-categoryListItem"):
        temp = x.get_attribute('value')
        if temp not in event_types and temp != None:
            event_types.append(temp)
    return event_types


In [3]:
def get_events(driver, event_types, saved_tab):
    driver.switch_to.window(saved_tab)
    allEvents = {}
    for item in event_types:
        event_block = driver.find_element(By.XPATH, "//div[@value='"+item+"']")
        event_block.click()
        # Switch to over/under
        # over_under = event_block.find_element(By.CSS_SELECTOR, "[value='over_under']")
        over_under = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//li[.='Over/Under']")))
        over_under.click()
        # switch to over/under 2.5
        over_under_25 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//li[text()='TG 2.5']")))
        over_under_25.click()
        leagues = {}
        l_count = driver.find_elements(By.CLASS_NAME, "BLM-leagueName")
        div_index = 3
        for league_index in range(len(l_count)):
            league = driver.find_element(By.XPATH, "(//div[@class='BLM-leagueName'])[position()="+str(league_index+1)+"]")
            print(str(league.text))
            # Set first leagues div index
            # Capture each event in the league for today
            # First check number of events in this league
            league_item_count = driver.find_elements(By.XPATH, "//div[@id='BLM-sports-topLeagues']/div["+str(div_index)+"]//div[@class='BLM-leagueBox-content']/div")
            for event_index in range(1,len(league_item_count)+1):
                str_event = ""
                for part in range(1,3):
                    event = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@id='BLM-sports-topLeagues']/div["+str(div_index)+"]//div[@class='BLM-leagueBox-content']/div["+str(event_index)+"]//div[@class='BLM-matchDetails']/div/div/div["+str(part)+"]")))
                    str_event += str(event.text)
                    if part == 1:
                        str_event += ' vs '
                print('\t'+str_event)
            div_index+=1
        quit()
        


In [4]:
driver = webdriver.Chrome(var.driver_path)
driver.maximize_window()


In [5]:
# Loop through available sites and get events
for index, item in enumerate(var.sites.keys()):
    if item == 'yetu':
        var.events[item] = scrape_event_type(driver, var.sites[item])
    # elif item == 'mel':
    #     var.events[item] = melbet.scrape_event_type(driver, var.sites[item])
    # elif item == 'sporty':
    #     var.events[item] = sporty.scrape_event_type(driver, var.sites[item])
    else:
        print(f'Unknown site index {item}')

    # Check if tab for site saved in tabs
    if item not in var.tabs.keys():
        var.tabs[item] = driver.window_handles[index]
    if index < len(var.sites)-1:
        # Open new tab
        driver.execute_script("window.open('')")
        driver.switch_to.window(driver.window_handles[index+1])


get_events(driver, var.events['yetu'], var.tabs['yetu'])


Champions League
	Inter Milan vs Shakhtar
	Besiktas vs Ajax
	Man City vs Paris SG
	Liverpool vs Porto
	Club Brugge vs RB Leipzig
	Atletico Madrid vs Milan
	Sheriff Tiraspol vs Real Madrid
	Sporting CP vs Borussia Dortmund
English Championship
	Barnsley vs Swansea
	Millwall vs Bournemouth
	Fulham vs Derby
	Cardiff vs Hull
	Blackburn vs Peterborough
	QPR vs Huddersfield
	Bristol City vs Stoke
German Bundesliga 2
	St.Pauli vs SV Sandhausen
Europa League
	Lokomotiv Moscow vs Lazio
	Bayer Leverkusen vs Celtic
	Rapid Vienna vs West Ham
	Red Star Belgrade vs Ludogorets Razgrad
	FC Midtjylland vs Braga
	Betis vs Ferencvarosi TC
	Galatasaray vs Marseille
	Dinamo Zagreb vs KRC Genk
	Olympiakos vs Fenerbahçe
	Leicester vs Legia Warsaw
	Eintracht Frankfurt vs Antwerpen
	Brøndby IF vs Lyon
	PSV vs Sturm Graz
	Monaco vs Real Sociedad
	Rangers vs Sparta Prague


ElementClickInterceptedException: Message: element click intercepted: Element <div class="BLM-categoryListItem" value="tennis" onclick="getSportPageTopLeague(event)">...</div> is not clickable at point (418, 751). Other element would receive the click: <body>...</body>
  (Session info: chrome=96.0.4664.45)
